In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('../../data/illinois_ambient_annual_loads_wrtdsk_raw.csv', dtype={'site_no':str}).rename({'_flow_days':'flow_days'}, axis=1)

#df['year'] = pd.to_datetime(df['year'], format='%Y').values
#df = df.set_index(['site_no','year'])

df.head()

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,01045_n,80154_n,99220_n,flow_days
0,03336645,1979,4.441585e+08,32767808.21,5247228.68,NaN,2389157.45,3804656.32,210173.74,NaN,...,12,12,12,0,0,12,12,0,0,365
1,03336645,1980,2.823931e+08,22168364.84,2893754.65,NaN,1028470.86,2028750.03,88583.75,NaN,...,13,13,5,0,0,13,13,0,13,366
2,03336645,1981,1.886789e+08,21083359.36,4858619.78,NaN,977276.77,3475216.04,NaN,38629.77,...,12,9,0,9,0,12,12,0,12,365
3,03336645,1982,5.648412e+08,50349647.30,6185522.08,NaN,2942128.68,4068385.10,NaN,85213.76,...,8,8,0,8,0,8,8,0,8,365
4,03336645,1983,9.445388e+07,9443095.59,3856953.07,NaN,730680.37,3071834.27,NaN,31662.84,...,8,8,0,8,0,8,8,0,8,365


In [3]:
# VALUE LOOKS ACCEPTABLE
# data cleaning 00535_kg
param='00535'
site = '03384450'
years = [2015]

bad_index = (df.site_no == site) & (df.year.isin(years))
#df.loc[bad_index,[f'{param}_kg',f'{param}_gse']] = np.nan
df.loc[bad_index]

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,01045_n,80154_n,99220_n,flow_days
510,03384450,2015,4374649.7,434159.07,62897.72,NaN,NaN,14061.75,49317.85,NaN,...,0,11,11,0,0,9,9,0,9,365


In [4]:
# data cleaning for 00625_kg
param='00625'
site = '03382100'
years = [2018]
#years = np.arange(2000,2020)
bad_index = (df.site_no == site) & (df.year.isin(years))
df.loc[bad_index,[f'{param}_kg',f'{param}_gse']] = np.nan
df.loc[bad_index]

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,01045_n,80154_n,99220_n,flow_days
470,03382100,2018,17523855.0,1817083.98,192064.74,12368.43,NaN,54063.06,31419.5,NaN,...,9,9,9,0,0,9,9,0,9,365


In [5]:
df.max()

site_no                 05599490
year                        2021
00530_kg      34783839624.459999
00535_kg          31866576088.91
00600_kg            747691600.03
00610_kg             26079264.47
00625_kg            181181914.02
00630_kg            588420312.08
00665_kg              52012238.3
00666_kg             24238982.58
00667_kg             33804610.34
00946_kg           4854517168.21
01045_kg     509742697021.919983
80154_kg      22663451488.650002
99220_kg           2621663181.05
00530_gse                   3.22
00535_gse                    2.6
00600_gse                   1.52
00610_gse                   2.27
00625_gse                   2.23
00630_gse                   1.86
00665_gse                   2.13
00666_gse                   2.05
00667_gse                   2.15
00946_gse                   1.86
01045_gse                   2.45
80154_gse                   2.22
99220_gse                   1.48
00530_n                       26
00535_n                       22
00600_n   

In [6]:
import numpy as np
np.sort(df.year.unique())

array([1945, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1964, 1965,
       1966, 1967, 1968, 1969, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [7]:
# drop years prior to 1976
df = df[df['year'] >= 1976]

In [8]:
df = df.rename({'site_no':'site'}, axis=1).set_index(['site','year'])

In [9]:
# unit conversions
df['01045_kg'] = df['01045_kg'] * 0.001 # ug to mg

In [10]:
df.max()

00530_kg     3.478384e+10
00535_kg     3.186658e+10
00600_kg     7.476916e+08
00610_kg     2.607926e+07
00625_kg     1.811819e+08
00630_kg     5.884203e+08
00665_kg     5.201224e+07
00666_kg     2.423898e+07
00667_kg     3.380461e+07
00946_kg     4.854517e+09
01045_kg     5.097427e+08
80154_kg     2.266345e+10
99220_kg     2.621663e+09
00530_gse    3.220000e+00
00535_gse    2.600000e+00
00600_gse    1.520000e+00
00610_gse    2.270000e+00
00625_gse    2.230000e+00
00630_gse    1.860000e+00
00665_gse    2.130000e+00
00666_gse    2.050000e+00
00667_gse    2.150000e+00
00946_gse    1.860000e+00
01045_gse    2.450000e+00
80154_gse    2.220000e+00
99220_gse    1.480000e+00
00530_n      2.600000e+01
00535_n      2.200000e+01
00600_n      4.300000e+01
00610_n      2.600000e+01
00625_n      4.600000e+01
00630_n      1.050000e+02
00665_n      5.500000e+01
00666_n      4.600000e+01
00667_n      4.600000e+01
00946_n      4.300000e+01
01045_n      2.600000e+01
80154_n      1.640000e+02
99220_n     

In [11]:
#df.groupby('site').min()['year'].tolist()

# rename columns

In [12]:
df.columns = [col.replace('_n','_count') for col in df.columns]
units = [col.split('_')[1] for col in df.columns]
params = [col.split('_')[0] for col in df.columns] #if col.split('_')[0]!='flow
#params

In [13]:
mapper = {
          'site_no':'site',
          'year' : 'year',
          '00530':'total suspended solids',
          '00535':'volatile suspended solids',
          '00600':'total nitrogen',
          '00610':'ammonia nitrogen',
          '00625':'organic nitrogen',
          '00630':'nitrate nitrogen',
          '00665':'total phosphorus',
          '00666':'dissolved phosphorus',
          '00667':'particulate phosphorus',
          '00946':'sulfate',
          '80154':'suspended sediment',
          '99220':'chloride',
          '01045':'iron',
          'flow':'flow'}

In [14]:
col_names = []
for i, col in enumerate(df.columns):
    #print(col)
    name = mapper.get(col.split('_')[0])
    unit = units[i]
    col_names.append(f'{name} [{unit}]')
    #col_names.append
    
#col_names
df.columns = col_names
df.head()

total suspended solids [kg]  volatile suspended solids [kg]  \
site     year                                                                
03336645 1979                 4.441585e+08                     32767808.21   
         1980                 2.823931e+08                     22168364.84   
         1981                 1.886789e+08                     21083359.36   
         1982                 5.648412e+08                     50349647.30   
         1983                 9.445388e+07                      9443095.59   

               total nitrogen [kg]  ammonia nitrogen [kg]  \
site     year                                               
03336645 1979           5247228.68                    NaN   
         1980           2893754.65                    NaN   
         1981           4858619.78                    NaN   
         1982           6185522.08                    NaN   
         1983           3856953.07                    NaN   

               organic nitrogen [kg]  nitrate nitrogen [kg]  \
site     year                                                 
03336645 1979             2389157.45             3804656.32   
         1980             1028470.86             2028750.03   
         1981              977276.77             3475216.04   
         1982             2942128.68             4068385.10   
         1983              730680.37             3071834.27   

               total phosphorus [kg]  dissolved phosphorus [kg]  \
site     year                                                     
03336645 1979              210173.74                        NaN   
         1980               88583.75                        NaN   
         1981                    NaN                   38629.77   
         1982                    NaN                   85213.76   
         1983                    NaN                   31662.84   

               particulate phosphorus [kg]  sulfate [kg]  ...  \
site     year                                             ...   
03336645 1979                          NaN   23595704.07  ...   
         1980                          NaN   13844281.19  ...   
         1981                          NaN   18884915.68  ...   
         1982                          NaN   27214168.66  ...   
         1983                          NaN   20351567.73  ...   

               organic nitrogen [count]  nitrate nitrogen [count]  \
site     year                                                       
03336645 1979                        12                        12   
         1980                        13                        13   
         1981                        12                         9   
         1982                         8                         8   
         1983                         8                         8   

               total phosphorus [count]  dissolved phosphorus [count]  \
site     year                                                           
03336645 1979                        12                             0   
         1980                         5                             0   
         1981                         0                             9   
         1982                         0                             8   
         1983                         0                             8   

               particulate phosphorus [count]  sulfate [count]  iron [count]  \
site     year                                                                  
03336645 1979                               0               12            12   
         1980                               0               13            13   
         1981                               0               12            12   
         1982                               0                8             8   
         1983                               0                8             8   

               suspended sediment [count]  chloride [count]  flow [days]  
site     year                           

In [15]:
df.columns

Index(['total suspended solids [kg]', 'volatile suspended solids [kg]',
       'total nitrogen [kg]', 'ammonia nitrogen [kg]', 'organic nitrogen [kg]',
       'nitrate nitrogen [kg]', 'total phosphorus [kg]',
       'dissolved phosphorus [kg]', 'particulate phosphorus [kg]',
       'sulfate [kg]', 'iron [kg]', 'suspended sediment [kg]', 'chloride [kg]',
       'total suspended solids [gse]', 'volatile suspended solids [gse]',
       'total nitrogen [gse]', 'ammonia nitrogen [gse]',
       'organic nitrogen [gse]', 'nitrate nitrogen [gse]',
       'total phosphorus [gse]', 'dissolved phosphorus [gse]',
       'particulate phosphorus [gse]', 'sulfate [gse]', 'iron [gse]',
       'suspended sediment [gse]', 'chloride [gse]',
       'total suspended solids [count]', 'volatile suspended solids [count]',
       'total nitrogen [count]', 'ammonia nitrogen [count]',
       'organic nitrogen [count]', 'nitrate nitrogen [count]',
       'total phosphorus [count]', 'dissolved phosphorus [count]',

# load flow data

In [16]:
#df = df.reset_index()

flow_df = pd.read_csv('../../data/illinois_ambient_annual_flow.csv', dtype={'site_no':str}).rename({'site_no':'site'}, axis=1).set_index(['site','year'])
flow_df = flow_df.rename({'mean flow [cms]':'mean flow [m^3/sec]'}, axis=1)
out = df.merge(flow_df, left_index=True, right_index=True, how='left')
#out = df.merge(flow_df, on=['site','year'], how='left')
#out = out.set_index(['site','year'])

out.head()

total suspended solids [kg]  volatile suspended solids [kg]  \
site     year                                                                
03336645 1979                 4.441585e+08                     32767808.21   
         1980                 2.823931e+08                     22168364.84   
         1981                 1.886789e+08                     21083359.36   
         1982                 5.648412e+08                     50349647.30   
         1983                 9.445388e+07                      9443095.59   

               total nitrogen [kg]  ammonia nitrogen [kg]  \
site     year                                               
03336645 1979           5247228.68                    NaN   
         1980           2893754.65                    NaN   
         1981           4858619.78                    NaN   
         1982           6185522.08                    NaN   
         1983           3856953.07                    NaN   

               organic nitrogen [kg]  nitrate nitrogen [kg]  \
site     year                                                 
03336645 1979             2389157.45             3804656.32   
         1980             1028470.86             2028750.03   
         1981              977276.77             3475216.04   
         1982             2942128.68             4068385.10   
         1983              730680.37             3071834.27   

               total phosphorus [kg]  dissolved phosphorus [kg]  \
site     year                                                     
03336645 1979              210173.74                        NaN   
         1980               88583.75                        NaN   
         1981                    NaN                   38629.77   
         1982                    NaN                   85213.76   
         1983                    NaN                   31662.84   

               particulate phosphorus [kg]  sulfate [kg]  ...  \
site     year                                             ...   
03336645 1979                          NaN   23595704.07  ...   
         1980                          NaN   13844281.19  ...   
         1981                          NaN   18884915.68  ...   
         1982                          NaN   27214168.66  ...   
         1983                          NaN   20351567.73  ...   

               nitrate nitrogen [count]  total phosphorus [count]  \
site     year                                                       
03336645 1979                        12                        12   
         1980                        13                         5   
         1981                         9                         0   
         1982                         8                         0   
         1983                         8                         0   

               dissolved phosphorus [count]  particulate phosphorus [count]  \
site     year                                                                 
03336645 1979                             0                               0   
         1980                             0                               0   
         1981                             9                               0   
         1982                             8                               0   
         1983                             8                               0   

               sulfate [count]  iron [count]  suspended sediment [count]  \
site     year                                                              
03336645 1979               12            12                           0   
         1980               13            13                           0   
         1981               12            12                           0   
         1982                8             8                           0   
         1983                8             8                           0   

               chloride [count]  flow [days]  mean flow [m^3/sec]  
site     year                    

# compute metadata

In [17]:
import xarray as xr
from meta import *

In [18]:
#df = df.set_index(['site','datetime'])
ds = out.to_xarray()
ds

<xarray.Dataset>
Dimensions:                            (site: 83, year: 46)
Coordinates:
  * site                               (site) object '03336645' ... '05599490'
  * year                               (year) int64 1976 1977 1978 ... 2020 2021
Data variables: (12/41)
    total suspended solids [kg]        (site, year) float64 nan ... 1.439e+08
    volatile suspended solids [kg]     (site, year) float64 nan ... 1.895e+07
    total nitrogen [kg]                (site, year) float64 nan ... 2.622e+06
    ammonia nitrogen [kg]              (site, year) float64 nan ... 2.155e+05
    organic nitrogen [kg]              (site, year) float64 nan nan ... 1.91e+06
    nitrate nitrogen [kg]              (site, year) float64 nan ... 7.047e+05
    ...                                 ...
    sulfate [count]                    (site, year) float64 nan nan ... 4.0 4.0
    iron [count]                       (site, year) float64 nan nan ... 4.0 4.0
    suspended sediment [count]         (site, year) float64 nan nan ... 0.0 0.0
    chloride [count]                   (site, year) float64 nan nan ... 4.0 4.0
    flow [days]                        (site, year) float64 nan nan ... 365.0
    mean flow [m^3/sec]                (site, year) float64 nan nan ... 69.65

In [19]:
#meta_file = 'https://www.sciencebase.gov/catalog/file/get/60abcec0d34ea221ce51e8a3?f=__disk__92%2F30%2Fb7%2F9230b7cfe0ad43e5eb1e36353a135d0bb6d824be'
meta_file = 'illinois_ambient_annual_loads_wrtdsk_metadata.xml'
import_metadata(ds, meta_file)

In [20]:
update_metadata(ds)

In [21]:
out = ds.to_dataframe()

In [22]:
out = out.dropna(how='all')
#out.reset_index().year.value_counts().sort_index()

In [23]:
out.to_csv('../../data/illinois_ambient_annual_loads_wrtdsk.csv')

In [24]:
dump_metadata(ds, '../../data/llinois_ambient_annual_loads_wrtdsk_metadata.xml')

# Scratch

In [25]:
import dataretrieval

In [32]:
from dataretrieval import nwis
df, meta = nwis.get_qwdata(sites='03346000', wide_format=False)

In [41]:
test = df[df.parm_cd==630]

In [42]:
test

,agency_cd,site_no,sample_end_dt,sample_end_tm,tm_datum_rlbty_cd,coll_ent_cd,medium_cd,project_cd,aqfr_cd,tu_id,...,dqi_cd,rpt_lev_va,rpt_lev_cd,lab_std_va,prep_set_no,prep_dt,anl_set_no,anl_dt,result_lab_cm_tx,anl_ent_cd
datetime,,,,,,,,,,,,,,,,,,,,,
1978-07-25 16:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978-09-05 22:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978-09-25 22:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978-11-01 21:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978-12-04 22:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996-10-08 14:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-12-09 20:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-01-24 13:00:00+00:00,USGS,03346000,NaN,NaN,T,IL-EPA,WS,NaN,NaN,NaN,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
